In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Courses/Fall 2021/dlsys/bnn-cf-vs-robust


# Create data

In [ ]:
# from requirements of https://github.com/hendrycks/robustness/tree/master/ImageNet-C/imagenet_c
# for natural corruptions 
%%capture
!apt install libmagickwand-dev
!pip install numpy Pillow scikit-image Wand opencv-python scipy # most likely only Wand is needed 

In [ ]:
import os
from pathlib import Path

import numpy as np
from tqdm.notebook import tqdm
import yaml 

import torch
import torchvision

from src import monochrome_corruptions 
from src.pmnist_robustness_data_utils import create_and_save_datasets


In [ ]:
data_raw_path = Path('data/raw')
data_inp_path = Path('data/input')

mnist_data_path = data_raw_path / 'mnist_pytorch'
root_data_path = data_inp_path / 'pmnist_robustness_test'

task_sequence = ['MNIST', 'pMNIST']

task_metadata = dict(
    task_sequence = task_sequence,
    num_tasks = len(task_sequence),
    root_data = str(root_data_path), 
    task_ids = ['task-%02d' %(i+1) for i in range(len(task_sequence))],
    task_paths = dict()
)

In [ ]:
%%time 
# ignore warning about `true_divide`, this was fixed by replacing these inf or nan values with 0 
root_data_path.mkdir(parents=True, exist_ok=True)

subset_mono_natcrpt = {k: v for i, (k,v) in 
                       enumerate(monochrome_corruptions.corruption_collection.items())
                       if i < 2}

                    
perturbation_actions = dict(
    type='corruptions',
    fns=subset_mono_natcrpt
)

task_ids = task_metadata['task_ids']
for i  in tqdm(range(len(task_ids))):
    task_id = task_ids[i]
    task_name = task_metadata['task_sequence'][i]
    perm_dset = 'p' in  task_name

    task_metadata['task_paths'][task_id] = create_and_save_datasets(
        mnist_data_path, 
        root_data_path, 
        task_id, 
        perm=perm_dset, 
        actions=perturbation_actions
    )

with open(root_data_path / 'metadata.yaml', 'w') as file:
    documents = yaml.safe_dump(task_metadata, file, default_flow_style=False)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/raw/mnist_pytorch/MNIST/raw/train-images-idx3-ubyte.gz to data/raw/mnist_pytorch/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/raw/mnist_pytorch/MNIST/raw/train-labels-idx1-ubyte.gz to data/raw/mnist_pytorch/MNIST/raw



...

  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/raw/mnist_pytorch/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/raw/mnist_pytorch/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# Run training and testing

In [ ]:
!python src/pmnist_robustness_main.py --help

usage: pmnist_robustness_main.py [-h] [--input INPUT]
                                 [--batch-size BATCH_SIZE]
                                 [--num-workers NUM_WORKERS] [--pin-memory]
                                 [--hidden-layers HIDDEN_LAYERS [HIDDEN_LAYERS ...]]
                                 [--init INIT] [--init-width INIT_WIDTH]
                                 [--lr LR] [--meta META [META ...]]
                                 [--decay DECAY] [--gamma GAMMA]
                                 [--epochs-per-task EPOCHS_PER_TASK]
                                 [--save-path SAVE_PATH]
                                 [--output-name OUTPUT_NAME] [--seed SEED]
                                 [--device DEVICE]

BNN learning several tasks in a row, metaplasticity is controlled by the
argument meta.

optional arguments:
  -h, --help            show this help message and exit
  --input INPUT         Source for meta-data of task sequence
  --batch-size BATCH_SIZE
              

In [ ]:
torch.__version__

'1.9.0+cu111'

In [ ]:
%%time 
!python -W ignore src/pmnist_robustness_main.py \
    --input data/input/pmnist_robustness_test/metadata.yaml \
    --hidden-layers 256 256 \
    --meta 1.25 \
    --decay 1e-7 \
    --epochs-per-task 1 \
    --batch-size 200 \
    --num-workers 1 \
    --output-name pmnist_robustness_test


>>>>>>>>>>>>>>>>>>>> EXPERIMENT CONFIG <<<<<<<<<<<<<<<<<<

areas:
- null
batch_size: 200
beaker: false
bin_path: false
decay: 1.0e-07
device: 0
epochs_per_task: 1
ewc: false
ewc_lambda: 0.0
fb: null
gamma: 1.0
hidden_layers:
- 256
- 256
in_size: 784
init: uniform
init_width: 0.1
input: data/input/pmnist_robustness_test/metadata.yaml
interleaved: false
lr: 0.005
meta:
- 1.25
n_bk: null
net: bnn
norm: bn
num_workers: 1
out_size: 10
output_name: pmnist_robustness_test
pin_memory: false
ratios:
- null
rnd_consolidation: false
save_path: ./data/output
scenario: task
seed: null
si: false
si_lambda: 0.0

>>>>>>>>>>>>>>>>>>>> MODEL CONSTRUCTION <<<<<<<<<<<<<<<<<<

BNN(
  (layers): ModuleDict(
    (fc1): BinarizeLinear(in_features=784, out_features=256, bias=False)
    (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc2): BinarizeLinear(in_features=256, out_features=256, bias=False)
    (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, tr

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv('data/output/pmnist_robustness_test/perf_forget.csv')

,net,arch,lr,meta,task_order,task_id,task_epoch,glob_epoch,train_acc,train_loss,train_time,test_acc::task-01,test_loss::task-01,test_time::task-01,test_acc::task-02,test_loss::task-02,test_time::task-02
0,bnn,256-256,0.005,"{'l1': 1.25, 'l2': 1.25, 'l3': 1.25}",1,task-01,1,1,95.44,0.19812,8.466213,94.93,0.211207,1.222375,10.00,3.731435,1.275142
1,bnn,256-256,0.005,"{'l1': 1.25, 'l2': 1.25, 'l3': 1.25}",2,task-02,1,2,95.53,0.16448,8.576909,29.78,2.266501,1.229753,94.78,0.183985,1.203748


In [ ]:
pd.read_csv('data/output/pmnist_robustness_test/perf_robust.csv')


,source_task,data_key,train_phase,test_acc,test_loss,test_time
0,task-01,corruptions::gaussian_noise,task-01,22.07,2.591809,1.243476
1,task-01,corruptions::gaussian_noise,task-02,25.28,3.005815,1.236060
2,task-01,corruptions::shot_noise,task-01,29.86,2.266211,1.202126
3,task-01,corruptions::shot_noise,task-02,32.19,2.534090,1.258311
4,task-01,original::test,task-01,29.78,2.266501,1.186190
5,task-01,original::test,task-02,32.02,2.518642,1.266876
6,task-02,corruptions::gaussian_noise,task-01,72.95,0.816823,1.251940
7,task-02,corruptions::gaussian_noise,task-02,92.94,0.246275,1.200336
8,task-02,corruptions::shot_noise,task-01,79.17,0.644311,1.205440
9,task-02,corruptions::shot_noise,task-02,94.88,0.184223,1.216775
